In [2]:
#2020-ml-p4-re
!pip uninstall --y kaggle 
!pip install --upgrade pip
!pip install kaggle==1.5.6

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v


Uninstalling kaggle-1.5.9:
  Successfully uninstalled kaggle-1.5.9
     |████████████████████████████████| 1.5MB 9.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 4.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=177a79854984ce3e156fb3b738427c9ca6dfcb38e0d6a0157da106fe9e1839a6
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Kaggle API 1.5.6


In [1]:
!kaggle competitions download -c 2020ml4re
!unzip 2020ml4re.zip

2020ml4re.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2020ml4re.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
import seaborn as sns
import pandas as pd 
import numpy as np 

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


df_train.drop(['age_group','religion'], axis=1, inplace=True)
df_test.drop(['age_group','religion'], axis=1, inplace=True)

full_data = [df_train,df_test]
pd.options.display.max_columns = None
print(df_train.head(20))
print(df_test.head(20)) 

    index  QaA   QaE  QbA    QbE  QcA    QcE  QdA   QdE  QeA   QeE  QfA   QfE  \
0       2  4.0  1623  1.0   1480  1.0   1021  4.0  3374  5.0  1333  1.0   531   
1       3  3.0   504  3.0   2311  4.0    992  3.0  3245  1.0   357  2.0  1519   
2       5  2.0   834  1.0   1769  4.0   2105  1.0  1070  5.0  1930  2.0  1258   
3       8  5.0   795  2.0   3469  4.0   1693  3.0  1991  4.0  5922  2.0  1322   
4       9  2.0  1668  1.0    866  1.0    895  1.0  1308  2.0  1715  5.0   563   
5      15  2.0   447  4.0   1316  4.0    745  1.0   526  3.0  1121  1.0   637   
6      16  1.0   390  5.0   1322  5.0   1947  1.0   887  1.0  1127  2.0   978   
7      18  3.0   359  5.0    603  4.0    483  1.0   555  2.0   379  2.0   331   
8      19  2.0   989  5.0   5602  5.0    542  2.0  2067  2.0  1004  2.0   615   
9      20  3.0   309  4.0    877  5.0    628  3.0   427  5.0   726  4.0   664   
10     21  2.0   619  4.0   1012  5.0    597  1.0   936  1.0  2443  5.0  1552   
11     22  4.0   366  2.0   

In [4]:
print (df_train[["gender", "voted"]].groupby(['gender'], as_index=False).mean())
print (df_train[["race", "voted"]].groupby(['race'], as_index=False).mean())

for dataset in full_data:
    dataset['gender'] = dataset['gender'].map( {'Female': 0, 'Male': 1} ).astype(int)
    dataset['race'] = dataset['race'].map( {'Arab': 0, 'Asian': 1,'Black': 2, 'Indigenous Australian': 3,'Native American':4,'Other':5,'White':6}).astype(int)

   gender     voted
0  Female  1.542350
1    Male  1.557257
                    race     voted
0                   Arab  1.746032
1                  Asian  1.705830
2                  Black  1.620388
3  Indigenous Australian  1.520000
4        Native American  1.616487
5                  Other  1.606307
6                  White  1.500261


In [5]:
X = df_train.iloc[:,1:-1]
y = df_train.iloc[:,[-1]]
test_x = df_test.iloc[:,1:]
print(X.dtypes)
print(y.shape)
print(test_x.shape)
#religion    object city 
#gender         object male , female
#age_group      object 30s...
#race           object white .. 

QaA      float64
QaE        int64
QbA      float64
QbE        int64
QcA      float64
          ...   
wr_09      int64
wr_10      int64
wr_11      int64
wr_12      int64
wr_13      int64
Length: 74, dtype: object
(22310, 1)
(9562, 74)


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=777,stratify=y)

In [17]:
print('y_train class distribution')
print(y_train.value_counts(normalize=True))

print('y_test class distribution')
print(y_test.value_counts(normalize=True))

y_train class distribution
voted
2        0.550263
1        0.449737
dtype: float64
y_test class distribution
voted
2        0.550376
1        0.449624
dtype: float64


In [26]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(test_x)
y_pred

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([2, 2, 1, ..., 2, 1, 2])

In [27]:
submit = pd.read_csv('sampleSubmission.csv')

submit

,index,voted
0,6,1
1,10,1
2,12,1
3,14,1
4,25,1
...,...,...
9557,45506,1
9558,45509,1
9559,45512,1
9560,45516,1


In [28]:
for i in range(len(y_pred)):
    submit['voted'][i] = y_pred[i].item()

submit

,index,voted
0,6,2
1,10,2
2,12,1
3,14,1
4,25,1
...,...,...
9557,45506,2
9558,45509,2
9559,45512,2
9560,45516,1


In [29]:
submit.to_csv('result.csv',mode='w',index=False)
!kaggle competitions submit -c 2020ml4re -f result.csv -m "14010974_이기택"

100% 72.5k/72.5k [00:00<00:00, 156kB/s]
Successfully submitted to 2020.ML.중간고사.문제4.다시풀기